In [ ]:
!unzip /content/Img.zip

In [ ]:

#Inspirado: https://colab.research.google.com/github/practicaldl/Practical-Deep-Learning-Book/blob/master/code/chapter-4/1-feature-extraction.ipynb#scrollTo=hJPLApcFQLDV
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm import tqdm, tqdm_notebook
import os
import random
import time
import math
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D


In [ ]:
def model_picker(name):
    if (name == 'vgg16'):
        model = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'vgg19'):
        model = VGG19(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'mobilenet'):
        model = MobileNet(weights='imagenet',
                          include_top=False,
                          input_shape=(224, 224, 3),
                          pooling='max',
                          depth_multiplier=1,
                          alpha=1)
    elif (name == 'inception'):
        model = InceptionV3(weights='imagenet',
                            include_top=False,
                            input_shape=(224, 224, 3),
                            pooling='max')
    elif (name == 'resnet'):
        model = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                        pooling='max')
    elif (name == 'xception'):
        model = Xception(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                         pooling='max')
    else:
        print("Specified model not available")
    return model

In [ ]:
def extract_features(img_path, model, classe):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path,
                         target_size=(input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    #print(normalized_features)
    normalized_features = np.append(normalized_features, classe)
    #print(normalized_features)

    return normalized_features

In [ ]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']

def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
    return file_list

# Gerando para todos os Deep Features

In [ ]:

for arquitetura in ['inception', 'vgg16', 'vgg19', 'resnet', 'mobilenet', 'xception']:
  print("Arquitetura ", arquitetura )
  model_architecture = arquitetura
  model = model_picker(model_architecture)

  normal = sorted(get_file_list('/content/deep_feature_extractor/Imagens/01_Normal'))
  covid_19 = sorted(get_file_list('/content/deep_feature_extractor/Imagens/02_COVID-19/'))

  print("Extraindo Caracteristicas!")
  feature_list = []
  for i in (range(len(normal))):
      feature_list.append(extract_features(normal[i], model, 0))
  
  features_normal = feature_list

  feature_list = []
  for i in (range(len(covid_19))):
      feature_list.append(extract_features(covid_19[i], model, 1))
  features_covid = feature_list

  dataset = np.concatenate((features_covid, features_normal))
  np.random.shuffle(dataset)

  print("Total de caracteristicas:", len( dataset[0]-1 ))
  print("Salvando em Treino_"+str(model_architecture)+".pickle")
  pickle.dump(
      dataset,
      open('dataset_' + model_architecture + '.pickle', 'wb'))
  print("=======")

Arquitetura  inception
Extraindo Caracteristicas!
Total de caracteristicas: 2049
Salvando em Treino_inception.pickle
Arquitetura  vgg16
Extraindo Caracteristicas!
Total de caracteristicas: 513
Salvando em Treino_vgg16.pickle
Arquitetura  vgg19
Extraindo Caracteristicas!
Total de caracteristicas: 513
Salvando em Treino_vgg19.pickle
Arquitetura  resnet
Extraindo Caracteristicas!
Total de caracteristicas: 2049
Salvando em Treino_resnet.pickle
Arquitetura  mobilenet
Extraindo Caracteristicas!
Total de caracteristicas: 1025
Salvando em Treino_mobilenet.pickle
Arquitetura  xception
83689472/83683744 [==============================] - 1s 0us/step
Extraindo Caracteristicas!
Total de caracteristicas: 2049
Salvando em Treino_xception.pickle


In [ ]:
!zip -r my_features.zip ./*

  adding: dataset_inception.pickle (deflated 51%)
  adding: dataset_mobilenet.pickle (deflated 69%)
  adding: dataset_resnet.pickle (deflated 54%)
  adding: dataset_vgg16.pickle (deflated 56%)
  adding: dataset_vgg19.pickle (deflated 55%)
  adding: dataset_xception.pickle (deflated 77%)


In [ ]:
#Abrir os arquivos Pickle

# open a file, where you stored the pickled data
file = open('/content/deep_feature_extractor/features/inception.pickle', 'rb')

# dump information to that file
data = pickle.load(file)